<a href="https://colab.research.google.com/github/pcmbs/preset-embedding_audio-model-selection/blob/main/notebooks_colab/preset_embedding_audio_model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone GitHub repo

In [1]:
!git clone -b cav-mae https://github.com/pcmbs/preset-embedding_audio-model-selection.git

# change current directory
%cd /content/preset-embedding_audio-model-selection

Cloning into 'preset-embedding_audio-model-selection'...
remote: Enumerating objects: 1334, done.
remote: Counting objects: 100% (585/585), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 1334 (delta 421), reused 435 (delta 294), pack-reused 749
Receiving objects: 100% (1334/1334), 122.46 MiB | 25.82 MiB/s, done.
Resolving deltas: 100% (733/733), done.
/content/preset-embedding_audio-model-selection


Download dependencies from requirements.txt

In [2]:
#@title
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... do

set PROJECT_ROOT python environment variable (for hydra config)

In [3]:
import os
os.environ["PROJECT_ROOT"] = "/content/preset-embedding_audio-model-selection"

# Download nsynth-lite
(only required for the nearest neighbors evaluation)

In [ ]:
#@title
!cd data && gdown "1720jJyIB1BAVoGg5Caq1SF78Gy4vZuEt" && unzip -q -o nsynth-lite.zip && rm nsynth-lite.zip && rm -r __MACOSX

Downloading...
From: https://drive.google.com/uc?id=1720jJyIB1BAVoGg5Caq1SF78Gy4vZuEt
To: /content/preset-embedding_audio-model-colab/data/nsynth-lite.zip
100% 1.65G/1.65G [00:15<00:00, 103MB/s]


# Download sound attributes ranking dataset
(only required for the sound attributes ranking evaluation)


In [4]:
!cd data/TAL-NoiseMaker && gdown "1f6Hkkveset6vwdbgvXImHmYTh79hBqv1" && unzip -q -o sound_attributes_ranking_dataset.zip && rm sound_attributes_ranking_dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1f6Hkkveset6vwdbgvXImHmYTh79hBqv1
To: /content/preset-embedding_audio-model-selection/data/TAL-NoiseMaker/sound_attributes_ranking_dataset.zip
100% 1.49G/1.49G [00:18<00:00, 81.1MB/s]


# Hydra Config

Run the following cell to print the current config

In [ ]:
#@title
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.hydra_config import HydraConfig

with initialize(version_base=None, config_path="configs", job_name="print_cfg"):
    cfg = compose(config_name="evaluate_audio_model", return_hydra_config=True)
    cfg.hydra.job.num = 0 # arbitrary value, otherwise raise an error since missing
    cfg.hydra.runtime.output_dir = cfg.hydra.run.dir # same as above, should be that for single run
    HydraConfig.instance().set_config(cfg)
    cfg_no_hydra = {k: v for k, v in cfg.items() if k != "hydra"} # dont print hydra config
    print(OmegaConf.to_yaml(cfg_no_hydra, resolve=True))


seed: 667
task_name: passt_s_swa_p16_128_ap476_base_embed_only
wandb:
  dir: /content/preset-embedding_audio-model-selection/logs/passt_s_swa_p16_128_ap476_base_embed_only/2023-08-04_13-58-21
  mode: online
  id: null
  anonymous: null
  project: preset-embedding_audio-model-selection
  tags: []
model:
  encoder:
    _target_: models.passt.passt_wrapper.PasstWrapper
    features: base
    arch: passt_s_swa_p16_128_ap476
    mode: embed_only
  name: passt_s_swa_p16_128_ap476_base_embed_only
  group: passt
eval:
  sound_attributes_ranking:
    root: /content/preset-embedding_audio-model-selection/data/TAL-NoiseMaker/sound_attributes_ranking_dataset
distance_fn: pairwise_manhattan_distance
reduce_fn: flatten
paths:
  root_dir: /content/preset-embedding_audio-model-selection
  data_dir: /content/preset-embedding_audio-model-selection/data
  log_dir: /content/preset-embedding_audio-model-selection/logs
  ckpt_dir: /content/preset-embedding_audio-model-selection/checkpoints
  output_dir: /co

# Evaluate audio models

evaluate with the current config

In [ ]:
!python src/evaluate_audio_model.py

Evaluate all models (with corresponding hparams)

In [ ]:
# Encodec
!python src/evaluate_audio_model.py -m model="encodec24khz,encodec48khz" distance_fn="pairwise_manhattan_distance" reduce_fn="glob(*time)"
# OpenL3
!python src/evaluate_audio_model.py -m model="glob(*6144, exclude=openl3_linear*)" distance_fn="pairwise_manhattan_distance" reduce_fn="glob(*, exclude=*channel)"
# PaSST
!python src/evaluate_audio_model.py -m model="glob(passt_s*)" distance_fn="pairwise_manhattan_distance" reduce_fn="global_avg_pool_time,flatten" ++model.encoder.features="base,base2level,base2levelmel"
# EfficientAT
!python src/evaluate_audio_model.py -m model="glob(mn*)" distance_fn="pairwise_manhattan_distance" reduce_fn="global_avg_pool_time,flatten"
# CAV-MAE
!python src/evaluate_audio_model.py -m model="glob(cavmae*)" distance_fn="pairwise_manhattan_distance" reduce_fn="glob(*, exclude=*channel)"

evaluate encodec models

In [ ]:
!python src/evaluate_audio_model.py -m model="encodec24khz,encodec48khz" distance_fn="pairwise_manhattan_distance" reduce_fn="glob(*time)"

evaluate openl3 models

In [ ]:
!python src/evaluate_audio_model.py -m model="glob(*6144, exclude=openl3_linear*)" distance_fn="pairwise_manhattan_distance" reduce_fn="glob(*, exclude=*channel)"

evaluate passt models

In [ ]:
!python src/evaluate_audio_model.py -m model="glob(passt_s*)" distance_fn="pairwise_manhattan_distance" reduce_fn="global_avg_pool_time,flatten" ++model.encoder.features="base,base2level,base2levelmel"

Evaluate mn models


In [ ]:
!python src/evaluate_audio_model.py -m model="glob(mn*)" distance_fn="pairwise_manhattan_distance" reduce_fn="global_avg_pool_time,flatten"

Evaluate CAV-MAE models

In [ ]:
!python src/evaluate_audio_model.py -m model="glob(cavmae*)" distance_fn="pairwise_manhattan_distance" reduce_fn="glob(*, exclude=*channel)"